In [28]:
import requests
import pandas as pd
import time
from urllib.parse import quote

api_key = /* Your API key  */

capital_cities_in_asia = [
    {"city": "Tokyo", "state": "Tokyo", "country": "Japan"},
    {"city": "Beijing", "state": "Beijing", "country": "China"},
    {"city": "Seoul", "state": "Seoul", "country": "South Korea"},
    {"city": "New Delhi", "state": "Delhi", "country": "India"},
    {"city": "Bangkok", "state": "Bangkok", "country": "Thailand"},
    {"city": "Kathmandu", "state": "Central Region", "country": "Nepal"},
    {"city": "Manila", "state": "National Capital Region", "country": "Philippines"},
    {"city": "Jakarta", "state": "Jakarta", "country": "Indonesia"},
    {"city": "Hanoi", "state": "Hanoi", "country": "Vietnam"},
    {"city": "Taipei", "state": "Taipei", "country": "Taiwan"},
    {"city": "Singapore", "state": "Singapore", "country": "Singapore"},
    {"city": "Kuala Lumpur", "state": "Kuala Lumpur", "country": "Malaysia"},
    {"city": "Yangon", "state": "Yangon", "country": "Myanmar"},
    {"city": "Islamabad", "state": "Islamabad", "country": "Pakistan"},
    {"city": "Tehran", "state": "Tehran", "country": "Iran"},
    {"city": "Ankara", "state": "Ankara", "country": "Turkey"},
    {"city": "Ashgabat", "state": "Ahal", "country": "Turkmenistan"},
    {"city": "Bishkek", "state": "Bishkek", "country": "Kyrgyzstan"},
    {"city": "Dushanbe", "state": "Dushanbe", "country": "Tajikistan"},
    {"city": "Tashkent", "state": "Toshkent Shahri", "country": "Uzbekistan"},
    {"city": "Astana", "state": "Astana", "country": "Kazakhstan"},
    {"city": "Nur-Sultan", "state": "Astana", "country": "Kazakhstan"},
    {"city": "Ulaanbaatar", "state": "Ulaanbaatar", "country": "Mongolia"},
    {"city": "Doha", "state": "Baladiyat ad Dawhah ", "country": "Qatar"},
    {"city": "Abu Dhabi", "state": "Abu Dhabi", "country": "United Arab Emirates"},
    {"city": "Bawshar", "state": "Muscat", "country": "Oman"},
    {"city": "Amman", "state": "Amman", "country": "Jordan"},
    {"city": "Baku", "state": "Baki", "country": "Azerbaijan"},
    {"city": "Yerevan - Vahakni", "state": "Yerevan", "country": "Armenia"},
    {"city": "Colombo", "state": "Western", "country": "Sri Lanka"},
    {"city": "Phnom Penh", "state": "Phnom Penh", "country": "Cambodia"},
    {"city": "Vientiane", "state": "Vientiane", "country": "Laos"},
    {"city": "Male", "state": "", "country": "Maldives"},
    {"city": "Nicosia", "state": "", "country": "Cyprus"},
    {"city": "Manama", "state": "Capital Governorate", "country": "Bahrain"},
    {"city": "Tbilisi", "state": "T'bilisi", "country": "Georgia"}
]

def get_aqi_data(city, state, country):
    url = f'https://api.airvisual.com/v2/city?city={quote(city)}&state={state}&country={country}&key={api_key}'

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        print(f"Results for {city}: {data}")
        current_pollution = data.get("data", {}).get("current", {}).get("pollution", {})
        if current_pollution:
            current_pollution["city"] = city
            current_pollution["state"] = state
            current_pollution["country"] = country
        return current_pollution
    except requests.exceptions.RequestException as err:
        print(f"Error getting data for {city}: {err}")
        return None

def get_all_aqi_data(cities):
    all_aqi_data = []

    for city_data in cities:
        city = city_data["city"]
        state = city_data["state"]
        country = city_data["country"]

        aqi_data = None
        retries = 0
        while not aqi_data and retries < 3:  # Retry up to 3 times
            aqi_data = get_aqi_data(city, state, country)
            if not aqi_data:
                retries += 1
                print(f"Retrying {city} ({retries}/3)...")
                time.sleep(2 ** retries)  # Exponential backoff

        if aqi_data:
            all_aqi_data.append(aqi_data)

        # Delay before making the next API request to avoid rate limiting
        time.sleep(1)  # Add a delay of 1 second between API requests

    return all_aqi_data

# Get all AQI data for the specified cities with retries
all_aqi_data = get_all_aqi_data(capital_cities_in_asia)

# Create a DataFrame from the collected data
df = pd.DataFrame(all_aqi_data)

# Define the Excel file path to save the data
excel_filename = 'real_time_aqi_data_asia.xlsx'

# Export the DataFrame to Excel
df.to_excel(excel_filename, index=False)

print(f"Data saved to {excel_filename}")
print(df)



Results for Tokyo: {'status': 'success', 'data': {'city': 'Tokyo', 'state': 'Tokyo', 'country': 'Japan', 'location': {'type': 'Point', 'coordinates': [139.69171, 35.6895]}, 'current': {'pollution': {'ts': '2024-05-09T18:00:00.000Z', 'aqius': 38, 'mainus': 'p2', 'aqicn': 10, 'maincn': 'p2'}, 'weather': {'ts': '2024-05-09T18:00:00.000Z', 'tp': 11, 'pr': 1017, 'hu': 48, 'ws': 3.09, 'wd': 290, 'ic': '01n'}}}}
Results for Beijing: {'status': 'success', 'data': {'city': 'Beijing', 'state': 'Beijing', 'country': 'China', 'location': {'type': 'Point', 'coordinates': [116.462153, 39.941674]}, 'current': {'pollution': {'ts': '2024-05-09T18:00:00.000Z', 'aqius': 82, 'mainus': 'p2', 'aqicn': 68, 'maincn': 'p1'}, 'weather': {'ts': '2024-05-09T18:00:00.000Z', 'tp': 20, 'pr': 1006, 'hu': 49, 'ws': 3, 'wd': 120, 'ic': '01n'}}}}
Results for Seoul: {'status': 'success', 'data': {'city': 'Seoul', 'state': 'Seoul', 'country': 'South Korea', 'location': {'type': 'Point', 'coordinates': [126.975961, 37.5646